In [ ]:
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/viz.html
import os 
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/")
print(os.getcwd())

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

In [ ]:
nora_wt_rep12_mcool = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_rep12_mcool = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
ren_wt_rep12_mcool = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
ren_iaa_rep12_mcool = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_rep12_mcool = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"

In [ ]:
import cooler
# to print which resolutions are stored in the mcool, use list_coolers
cooler.fileops.list_coolers(nora_wt_rep12_mcool)

# r500k plot

In [ ]:
### to load a cooler with a specific resolution use the following syntax:
nora_wt_rep12_mcool_r500k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/500000')
nora_iaa_rep12_mcool_r500k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/500000')
ren_wt_rep12_mcool_r500k = cooler.Cooler(f'{ren_wt_rep12_mcool}::resolutions/500000')
ren_iaa_rep12_mcool_r500k = cooler.Cooler(f'{ren_iaa_rep12_mcool}::resolutions/500000')
wn_bpk25_rep12_mcool_r500k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/500000')


In [ ]:
chromstarts = []
chromnames = nora_wt_rep12_mcool_r500k.chromnames
for i in chromnames:
    print(f'{i} : {nora_wt_rep12_mcool_r500k.extent(i)}')
    chromstarts.append(nora_wt_rep12_mcool_r500k.extent(i)[0])


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)


# Smoothing & Interpolation
nora_wt_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r500k.matrix(balance=True)[:],
                                nora_wt_rep12_mcool_r500k.matrix(balance=False)[:], cutoff=3, max_levels=7)
nora_wt_rep12_mcool_r500k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r500k_adcg)

nora_iaa_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r500k.matrix(balance=True)[:],
                                nora_iaa_rep12_mcool_r500k.matrix(balance=False)[:], cutoff=3, max_levels=7)
nora_iaa_rep12_mcool_r500k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r500k_adcg)

wn_bpk25_rep12_mcool_r500k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r500k.matrix(balance=True)[:],
                                wn_bpk25_rep12_mcool_r500k.matrix(balance=False)[:], cutoff=3, max_levels=7)
wn_bpk25_rep12_mcool_r500k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r500k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
figure, axes = plt.subplots(figsize=(12, 5), nrows=1, ncols=3, sharex=True, sharey=True)
figure.subplots_adjust(left=0.09, right=0.94, bottom=-0.3, top=1.35, wspace=0.08)
figure.suptitle('all_ chromosomes', y=0.96)
# ax1
ax1 = axes[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall')
ax1.set_title('Nora_WT_r500k', y=1.01)
ax1.set_xticks(chromstarts)
ax1.set_xticklabels(chromnames, rotation=45)
ax1.set_yticks(chromstarts)
ax1.set_yticklabels(chromnames)
ax1.xaxis.tick_bottom()
# ax2
ax2 = axes[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall')
ax2.set_title('Nora_IAA_r500k', y=1.01)
ax2.set_xticks(chromstarts)
ax2.set_xticklabels(chromnames, rotation=45)
ax2.set_yticks(chromstarts)
ax2.set_yticklabels(chromnames)
ax2.xaxis.tick_bottom()
# ax3
ax3 = axes[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall')
ax3.set_title('WN_BPK25_r500k', y=1.01)
ax3.set_xticks(chromstarts)
ax3.set_xticklabels(chromnames, rotation=45)
ax3.set_yticks(chromstarts)
ax3.set_yticklabels(chromnames)
ax3.xaxis.tick_bottom()

# for ax in axes:
#     format_ticks(ax)
#     ax.set_xlabel('position(Mb)')
# plt.colorbar(im3, ax=axes, fraction=0.014, label='(balance=True)', pad=0.02, aspect=22)
# # plt.savefig("Nora_Fig2A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r500k.fall.heatmap.pdf")
# # plt.savefig("Nora_Fig2A_use_Nora_WT_IAA_WN_BPK25.rep12merged.r500k.fall.heatmap.png")
plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)

chromsome = "chr1"
start = 10_000
end = nora_wt_rep12_mcool_r500k.chromsizes[chromsome]
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)

# Smoothing & Interpolation
nora_wt_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
nora_wt_rep12_mcool_r500k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r500k_adcg)

nora_iaa_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                nora_iaa_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
nora_iaa_rep12_mcool_r500k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r500k_adcg)

wn_bpk25_rep12_mcool_r500k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                wn_bpk25_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
wn_bpk25_rep12_mcool_r500k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r500k_adcg)

ren_wt_rep12_mcool_r500k_adcg = adaptive_coarsegrain(ren_wt_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                ren_wt_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
ren_wt_rep12_mcool_r500k_adcg_interp = interp_nan(ren_wt_rep12_mcool_r500k_adcg)

ren_iaa_rep12_mcool_r500k_adcg = adaptive_coarsegrain(ren_iaa_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                ren_iaa_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
ren_iaa_rep12_mcool_r500k_adcg_interp = interp_nan(ren_iaa_rep12_mcool_r500k_adcg)

# interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
fig, axs = plt.subplots(figsize=(20, 5), nrows=1, ncols=5, sharex=True, sharey=True)
fig.subplots_adjust(left=0.06, right=0.96, bottom=-0.3, top=1.35, wspace=0.08)
fig.suptitle('chr1', y=0.96)
# ax1
ax1 = axs[0]
im1 = ax1.matshow(nora_wt_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
ax1.set_title('Nora_WT_r500k', y=1.01)
ax1.set_ylabel('position(Mb)')
ax1.xaxis.tick_bottom()
# ax2
ax2 = axs[1]
im2 = ax2.matshow(nora_iaa_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
ax2.set_title('Nora_IAA_r500k', y=1.01)
ax2.xaxis.tick_bottom()
# ax3
ax3 = axs[2]
im3 = ax3.matshow(wn_bpk25_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
ax3.set_title('WN_BPK25_r500k', y=1.01)
ax3.xaxis.tick_bottom()

# ax4
ax4 = axs[3]
im4 = ax4.matshow(ren_iaa_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
ax4.set_title('Ren_IAA_r500k', y=1.01)
ax4.xaxis.tick_bottom()
# ax5
ax5 = axs[4]
im5 = ax5.matshow(ren_wt_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
ax5.set_title('Ren_WT_r500k', y=1.01)
ax5.xaxis.tick_bottom()
plt.colorbar(im5, ax=axs, fraction=0.014, label='(balance=True)', pad=0.01, aspect=13)

for ax in axs:
        format_ticks(ax)
        ax.set_xlabel('position(Mb)')

# plt.savefig("chrom_view.Nora_WN_Ren.r500k.chr1.fall.heatmap.pdf")
# plt.savefig("chrom_view.Nora_WN_Ren.r500k.chr1.fall.heatmap.png")
plt.show()


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)

chromsome_list = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chrX", "chrY"]
for chrom in chromsome_list:
    chromsome = chrom
    start = 1_000
    end = nora_wt_rep12_mcool_r500k.chromsizes[chromsome]
    plot_region = (chromsome, start, end)
    ax_limit_extents = (start, end, end, start)

    # Smoothing & Interpolation
    nora_wt_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                    nora_wt_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_wt_rep12_mcool_r500k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r500k_adcg)

    nora_iaa_rep12_mcool_r500k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                    nora_iaa_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_iaa_rep12_mcool_r500k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r500k_adcg)

    wn_bpk25_rep12_mcool_r500k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                    wn_bpk25_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    wn_bpk25_rep12_mcool_r500k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r500k_adcg)

    ren_wt_rep12_mcool_r500k_adcg = adaptive_coarsegrain(ren_wt_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                    ren_wt_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_wt_rep12_mcool_r500k_adcg_interp = interp_nan(ren_wt_rep12_mcool_r500k_adcg)

    ren_iaa_rep12_mcool_r500k_adcg = adaptive_coarsegrain(ren_iaa_rep12_mcool_r500k.matrix(balance=True).fetch(plot_region),
                                    ren_iaa_rep12_mcool_r500k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_iaa_rep12_mcool_r500k_adcg_interp = interp_nan(ren_iaa_rep12_mcool_r500k_adcg)

    # interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
    fig, axs = plt.subplots(figsize=(20, 5), nrows=1, ncols=5, sharex=True, sharey=True)
    fig.subplots_adjust(left=0.06, right=0.96, bottom=-0.3, top=1.35, wspace=0.08)
    fig.suptitle(f'{chrom} view', y=0.96)
    # ax1
    ax1 = axs[0]
    im1 = ax1.matshow(nora_wt_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
    ax1.set_title('Nora_WT_r500k', y=1.01)
    ax1.set_ylabel('position(Mb)')
    ax1.xaxis.tick_bottom()
    # ax2
    ax2 = axs[1]
    im2 = ax2.matshow(nora_iaa_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
    ax2.set_title('Nora_IAA_r500k', y=1.01)
    ax2.xaxis.tick_bottom()
    # ax3
    ax3 = axs[2]
    im3 = ax3.matshow(wn_bpk25_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
    ax3.set_title('WN_BPK25_r500k', y=1.01)
    ax3.xaxis.tick_bottom()

    # ax4
    ax4 = axs[3]
    im4 = ax4.matshow(ren_iaa_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
    ax4.set_title('Ren_IAA_r500k', y=1.01)
    ax4.xaxis.tick_bottom()
    # ax5
    ax5 = axs[4]
    im5 = ax5.matshow(ren_wt_rep12_mcool_r500k_adcg_interp*100000, vmin=0, vmax=500, cmap='fall', extent=ax_limit_extents)
    ax5.set_title('Ren_WT_r500k', y=1.01)
    ax5.xaxis.tick_bottom()
    plt.colorbar(im5, ax=axs, fraction=0.014, label='(balance=True)', pad=0.01, aspect=13)

    for ax in axs:
            format_ticks(ax)
            ax.set_xlabel('position(Mb)')

    plt.savefig(f"chrom_view.Nora_WN_Ren.r500k.{chrom}.fall.heatmap.pdf")
    plt.savefig(f"chrom_view.Nora_WN_Ren.r500k.{chrom}.fall.heatmap.png")
    plt.show()


In [ ]:
%%bash
mkdir chrom_view_r500k_for_all
mv chrom_view.Nora_WN_Ren.r500k.*.fall.heatmap.pdf ./chrom_view_r500k_for_all/
mv chrom_view.Nora_WN_Ren.r500k.*.fall.heatmap.png ./chrom_view_r500k_for_all/


# r200k plot

In [ ]:
### to load a cooler with a specific resolution use the following syntax:
nora_wt_rep12_mcool_r200k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/200000')
nora_iaa_rep12_mcool_r200k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/200000')
ren_wt_rep12_mcool_r200k = cooler.Cooler(f'{ren_wt_rep12_mcool}::resolutions/200000')
ren_iaa_rep12_mcool_r200k = cooler.Cooler(f'{ren_iaa_rep12_mcool}::resolutions/200000')
wn_bpk25_rep12_mcool_r200k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/200000')


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)

chromsome_list = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chrX", "chrY"]
for chrom in chromsome_list:
    chromsome = chrom
    start = 1_000
    end = nora_wt_rep12_mcool_r200k.chromsizes[chromsome]
    plot_region = (chromsome, start, end)
    ax_limit_extents = (start, end, end, start)

    # Smoothing & Interpolation
    nora_wt_rep12_mcool_r200k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r200k.matrix(balance=True).fetch(plot_region),
                                    nora_wt_rep12_mcool_r200k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_wt_rep12_mcool_r200k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r200k_adcg)

    nora_iaa_rep12_mcool_r200k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r200k.matrix(balance=True).fetch(plot_region),
                                    nora_iaa_rep12_mcool_r200k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_iaa_rep12_mcool_r200k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r200k_adcg)

    wn_bpk25_rep12_mcool_r200k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r200k.matrix(balance=True).fetch(plot_region),
                                    wn_bpk25_rep12_mcool_r200k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    wn_bpk25_rep12_mcool_r200k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r200k_adcg)

    ren_wt_rep12_mcool_r200k_adcg = adaptive_coarsegrain(ren_wt_rep12_mcool_r200k.matrix(balance=True).fetch(plot_region),
                                    ren_wt_rep12_mcool_r200k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_wt_rep12_mcool_r200k_adcg_interp = interp_nan(ren_wt_rep12_mcool_r200k_adcg)

    ren_iaa_rep12_mcool_r200k_adcg = adaptive_coarsegrain(ren_iaa_rep12_mcool_r200k.matrix(balance=True).fetch(plot_region),
                                    ren_iaa_rep12_mcool_r200k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_iaa_rep12_mcool_r200k_adcg_interp = interp_nan(ren_iaa_rep12_mcool_r200k_adcg)

    # interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
    fig, axs = plt.subplots(figsize=(20, 5), nrows=1, ncols=5, sharex=True, sharey=True)
    fig.subplots_adjust(left=0.06, right=0.96, bottom=-0.3, top=1.35, wspace=0.08)
    fig.suptitle(f'{chrom} view', y=0.96)
    # ax1
    ax1 = axs[0]
    im1 = ax1.matshow(nora_wt_rep12_mcool_r200k_adcg_interp*100000, vmin=0, vmax=200, cmap='fall', extent=ax_limit_extents)
    ax1.set_title('Nora_WT_r200k', y=1.01)
    ax1.set_ylabel('position(Mb)')
    ax1.xaxis.tick_bottom()
    # ax2
    ax2 = axs[1]
    im2 = ax2.matshow(nora_iaa_rep12_mcool_r200k_adcg_interp*100000, vmin=0, vmax=200, cmap='fall', extent=ax_limit_extents)
    ax2.set_title('Nora_IAA_r200k', y=1.01)
    ax2.xaxis.tick_bottom()
    # ax3
    ax3 = axs[2]
    im3 = ax3.matshow(wn_bpk25_rep12_mcool_r200k_adcg_interp*100000, vmin=0, vmax=200, cmap='fall', extent=ax_limit_extents)
    ax3.set_title('WN_BPK25_r200k', y=1.01)
    ax3.xaxis.tick_bottom()

    # ax4
    ax4 = axs[3]
    im4 = ax4.matshow(ren_iaa_rep12_mcool_r200k_adcg_interp*100000, vmin=0, vmax=200, cmap='fall', extent=ax_limit_extents)
    ax4.set_title('Ren_IAA_r200k', y=1.01)
    ax4.xaxis.tick_bottom()
    # ax5
    ax5 = axs[4]
    im5 = ax5.matshow(ren_wt_rep12_mcool_r200k_adcg_interp*100000, vmin=0, vmax=200, cmap='fall', extent=ax_limit_extents)
    ax5.set_title('Ren_WT_r200k', y=1.01)
    ax5.xaxis.tick_bottom()
    plt.colorbar(im5, ax=axs, fraction=0.014, label='(balance=True)', pad=0.01, aspect=13)

    for ax in axs:
            format_ticks(ax)
            ax.set_xlabel('position(Mb)')

    plt.savefig(f"chrom_view.Nora_WN_Ren.r200k.{chrom}.fall.heatmap.pdf")
    plt.savefig(f"chrom_view.Nora_WN_Ren.r200k.{chrom}.fall.heatmap.png")
    plt.show()


In [ ]:
%%bash
mkdir chrom_view_r200k_for_all
mv chrom_view.Nora_WN_Ren.r200k.*.fall.heatmap.pdf ./chrom_view_r200k_for_all/
mv chrom_view.Nora_WN_Ren.r200k.*.fall.heatmap.png ./chrom_view_r200k_for_all/


# r100k plot

In [ ]:
### to load a cooler with a specific resolution use the following syntax:
nora_wt_rep12_mcool_r100k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/100000')
nora_iaa_rep12_mcool_r100k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/100000')
ren_wt_rep12_mcool_r100k = cooler.Cooler(f'{ren_wt_rep12_mcool}::resolutions/100000')
ren_iaa_rep12_mcool_r100k = cooler.Cooler(f'{ren_iaa_rep12_mcool}::resolutions/100000')
wn_bpk25_rep12_mcool_r100k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/100000')


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)

chromsome_list = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chrX", "chrY"]
for chrom in chromsome_list:
    chromsome = chrom
    start = 1_000
    end = nora_wt_rep12_mcool_r100k.chromsizes[chromsome]
    plot_region = (chromsome, start, end)
    ax_limit_extents = (start, end, end, start)

    # Smoothing & Interpolation
    nora_wt_rep12_mcool_r100k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r100k.matrix(balance=True).fetch(plot_region),
                                    nora_wt_rep12_mcool_r100k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_wt_rep12_mcool_r100k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r100k_adcg)

    nora_iaa_rep12_mcool_r100k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r100k.matrix(balance=True).fetch(plot_region),
                                    nora_iaa_rep12_mcool_r100k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_iaa_rep12_mcool_r100k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r100k_adcg)

    wn_bpk25_rep12_mcool_r100k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r100k.matrix(balance=True).fetch(plot_region),
                                    wn_bpk25_rep12_mcool_r100k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    wn_bpk25_rep12_mcool_r100k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r100k_adcg)

    ren_wt_rep12_mcool_r100k_adcg = adaptive_coarsegrain(ren_wt_rep12_mcool_r100k.matrix(balance=True).fetch(plot_region),
                                    ren_wt_rep12_mcool_r100k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_wt_rep12_mcool_r100k_adcg_interp = interp_nan(ren_wt_rep12_mcool_r100k_adcg)

    ren_iaa_rep12_mcool_r100k_adcg = adaptive_coarsegrain(ren_iaa_rep12_mcool_r100k.matrix(balance=True).fetch(plot_region),
                                    ren_iaa_rep12_mcool_r100k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_iaa_rep12_mcool_r100k_adcg_interp = interp_nan(ren_iaa_rep12_mcool_r100k_adcg)

    # interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
    fig, axs = plt.subplots(figsize=(20, 5), nrows=1, ncols=5, sharex=True, sharey=True)
    fig.subplots_adjust(left=0.06, right=0.96, bottom=-0.3, top=1.35, wspace=0.08)
    fig.suptitle(f'{chrom} view', y=0.96)
    # ax1
    ax1 = axs[0]
    im1 = ax1.matshow(nora_wt_rep12_mcool_r100k_adcg_interp*100000, vmin=0, vmax=100, cmap='fall', extent=ax_limit_extents)
    ax1.set_title('Nora_WT_r100k', y=1.01)
    ax1.set_ylabel('position(Mb)')
    ax1.xaxis.tick_bottom()
    # ax2
    ax2 = axs[1]
    im2 = ax2.matshow(nora_iaa_rep12_mcool_r100k_adcg_interp*100000, vmin=0, vmax=100, cmap='fall', extent=ax_limit_extents)
    ax2.set_title('Nora_IAA_r100k', y=1.01)
    ax2.xaxis.tick_bottom()
    # ax3
    ax3 = axs[2]
    im3 = ax3.matshow(wn_bpk25_rep12_mcool_r100k_adcg_interp*100000, vmin=0, vmax=100, cmap='fall', extent=ax_limit_extents)
    ax3.set_title('WN_BPK25_r100k', y=1.01)
    ax3.xaxis.tick_bottom()

    # ax4
    ax4 = axs[3]
    im4 = ax4.matshow(ren_iaa_rep12_mcool_r100k_adcg_interp*100000, vmin=0, vmax=100, cmap='fall', extent=ax_limit_extents)
    ax4.set_title('Ren_IAA_r100k', y=1.01)
    ax4.xaxis.tick_bottom()
    # ax5
    ax5 = axs[4]
    im5 = ax5.matshow(ren_wt_rep12_mcool_r100k_adcg_interp*100000, vmin=0, vmax=100, cmap='fall', extent=ax_limit_extents)
    ax5.set_title('Ren_WT_r100k', y=1.01)
    ax5.xaxis.tick_bottom()
    plt.colorbar(im5, ax=axs, fraction=0.014, label='(balance=True)', pad=0.01, aspect=13)

    for ax in axs:
            format_ticks(ax)
            ax.set_xlabel('position(Mb)')

    plt.savefig(f"chrom_view.Nora_WN_Ren.r100k.{chrom}.fall.heatmap.pdf")
    plt.savefig(f"chrom_view.Nora_WN_Ren.r100k.{chrom}.fall.heatmap.png")
    plt.show()


In [ ]:
%%bash
mkdir chrom_view_r100k_for_all
mv chrom_view.Nora_WN_Ren.r100k.*.fall.heatmap.pdf ./chrom_view_r100k_for_all/
mv chrom_view.Nora_WN_Ren.r100k.*.fall.heatmap.png ./chrom_view_r100k_for_all/


# r50k plot

In [ ]:
### to load a cooler with a specific resolution use the following syntax:
nora_wt_rep12_mcool_r50k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/50000')
nora_iaa_rep12_mcool_r50k = cooler.Cooler(f'{nora_iaa_rep12_mcool}::resolutions/50000')
ren_wt_rep12_mcool_r50k = cooler.Cooler(f'{ren_wt_rep12_mcool}::resolutions/50000')
ren_iaa_rep12_mcool_r50k = cooler.Cooler(f'{ren_iaa_rep12_mcool}::resolutions/50000')
wn_bpk25_rep12_mcool_r50k = cooler.Cooler(f'{wn_bpk25_rep12_mcool}::resolutions/50000')


In [ ]:
### plot data in fall heatmap
import matplotlib
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# 设置为可编辑字体，默认的字体在AI中打开是路径
matplotlib.rcParams['pdf.fonttype'] = 42
# to plot ticks in terms of megabases we use the EngFormatter
# https://matplotlib.org/gallery/api/engineering_formatter.html

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x', rotation=45)

chromsome_list = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chrX", "chrY"]
for chrom in chromsome_list:
    chromsome = chrom
    start = 1_000
    end = nora_wt_rep12_mcool_r50k.chromsizes[chromsome]
    plot_region = (chromsome, start, end)
    ax_limit_extents = (start, end, end, start)

    # Smoothing & Interpolation
    nora_wt_rep12_mcool_r50k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r50k.matrix(balance=True).fetch(plot_region),
                                    nora_wt_rep12_mcool_r50k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_wt_rep12_mcool_r50k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r50k_adcg)

    nora_iaa_rep12_mcool_r50k_adcg = adaptive_coarsegrain(nora_iaa_rep12_mcool_r50k.matrix(balance=True).fetch(plot_region),
                                    nora_iaa_rep12_mcool_r50k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    nora_iaa_rep12_mcool_r50k_adcg_interp = interp_nan(nora_iaa_rep12_mcool_r50k_adcg)

    wn_bpk25_rep12_mcool_r50k_adcg = adaptive_coarsegrain(wn_bpk25_rep12_mcool_r50k.matrix(balance=True).fetch(plot_region),
                                    wn_bpk25_rep12_mcool_r50k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    wn_bpk25_rep12_mcool_r50k_adcg_interp = interp_nan(wn_bpk25_rep12_mcool_r50k_adcg)

    ren_wt_rep12_mcool_r50k_adcg = adaptive_coarsegrain(ren_wt_rep12_mcool_r50k.matrix(balance=True).fetch(plot_region),
                                    ren_wt_rep12_mcool_r50k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_wt_rep12_mcool_r50k_adcg_interp = interp_nan(ren_wt_rep12_mcool_r50k_adcg)

    ren_iaa_rep12_mcool_r50k_adcg = adaptive_coarsegrain(ren_iaa_rep12_mcool_r50k.matrix(balance=True).fetch(plot_region),
                                    ren_iaa_rep12_mcool_r50k.matrix(balance=False).fetch(plot_region), cutoff=3, max_levels=7)
    ren_iaa_rep12_mcool_r50k_adcg_interp = interp_nan(ren_iaa_rep12_mcool_r50k_adcg)

    # interpolation=['antialiased', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman']
    fig, axs = plt.subplots(figsize=(20, 5), nrows=1, ncols=5, sharex=True, sharey=True)
    fig.subplots_adjust(left=0.06, right=0.96, bottom=-0.3, top=1.35, wspace=0.08)
    fig.suptitle(f'{chrom} view', y=0.96)
    # ax1
    ax1 = axs[0]
    im1 = ax1.matshow(nora_wt_rep12_mcool_r50k_adcg_interp*100000, vmin=0, vmax=50, cmap='fall', extent=ax_limit_extents)
    ax1.set_title('Nora_WT_r50k', y=1.01)
    ax1.set_ylabel('position(Mb)')
    ax1.xaxis.tick_bottom()
    # ax2
    ax2 = axs[1]
    im2 = ax2.matshow(nora_iaa_rep12_mcool_r50k_adcg_interp*100000, vmin=0, vmax=50, cmap='fall', extent=ax_limit_extents)
    ax2.set_title('Nora_IAA_r50k', y=1.01)
    ax2.xaxis.tick_bottom()
    # ax3
    ax3 = axs[2]
    im3 = ax3.matshow(wn_bpk25_rep12_mcool_r50k_adcg_interp*100000, vmin=0, vmax=50, cmap='fall', extent=ax_limit_extents)
    ax3.set_title('WN_BPK25_r50k', y=1.01)
    ax3.xaxis.tick_bottom()

    # ax4
    ax4 = axs[3]
    im4 = ax4.matshow(ren_iaa_rep12_mcool_r50k_adcg_interp*100000, vmin=0, vmax=50, cmap='fall', extent=ax_limit_extents)
    ax4.set_title('Ren_IAA_r50k', y=1.01)
    ax4.xaxis.tick_bottom()
    # ax5
    ax5 = axs[4]
    im5 = ax5.matshow(ren_wt_rep12_mcool_r50k_adcg_interp*100000, vmin=0, vmax=50, cmap='fall', extent=ax_limit_extents)
    ax5.set_title('Ren_WT_r50k', y=1.01)
    ax5.xaxis.tick_bottom()
    plt.colorbar(im5, ax=axs, fraction=0.014, label='(balance=True)', pad=0.01, aspect=13)

    for ax in axs:
            format_ticks(ax)
            ax.set_xlabel('position(Mb)')

    plt.savefig(f"chrom_view.Nora_WN_Ren.r50k.{chrom}.fall.heatmap.pdf")
    plt.savefig(f"chrom_view.Nora_WN_Ren.r50k.{chrom}.fall.heatmap.png")
    plt.show()


In [ ]:
%%bash
mkdir chrom_view_r50k_for_all
mv chrom_view.Nora_WN_Ren.r50k.*.fall.heatmap.pdf ./chrom_view_r50k_for_all/
mv chrom_view.Nora_WN_Ren.r50k.*.fall.heatmap.png ./chrom_view_r50k_for_all/


In [ ]:
import os
import glob
import platform

os_name = platform.system()
# 指定目录路径
directory = './chrom_view_r500k_for_all'
# 构建文件路径的通配符
file_pattern = os.path.join(directory, 'chrom_view.Nora_WN_Ren.*.fall.heatmap.png')
# 使用 glob 模块获取匹配的文件路径列表
png_files = sorted(glob.glob(file_pattern))
print(len(png_files))
# 打印文件路径列表
for file in png_files:
    # print(file)
    if os_name == 'Windows':
        print(file.split('\\')[2])
    else:
        print(file.split('/')[2])


# add r500k plots to summary ppt

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

r500k_png_files = ["./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr1.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr2.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr3.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr4.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr5.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr6.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr7.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr8.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr9.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr10.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr11.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr12.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr13.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr14.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr15.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr16.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr17.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr18.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chr19.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chrX.fall.heatmap.png",
            "./chrom_view_r500k_for_all/chrom_view.Nora_WN_Ren.r500k.chrY.fall.heatmap.png",
]

# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = len(r500k_png_files)
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    # print(pi)
    slide = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
    # 插入文本框
    # text_box = slide.shapes.add_textbox(left, top, width, height)
    text_box = slide.shapes.add_textbox(Cm(0.05), Cm(0.05), Cm(12.65), Cm(0.5))
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_frame = text_box.text_frame
    if os_name == 'Linux':
        text_frame.text = r500k_png_files[i-1].split('/')[2]
    elif os_name == 'Windows':
        text_frame.text = r500k_png_files[i-1].split('\\')[2]
    # 设置字体大小和加粗
    text_font = text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True
    # PPT 页面指定区域插入图片
    # pic = slide.shapes.add_picture(r500k_png_files[i-1], left, top, width, height)
    pic = slide.shapes.add_picture(r500k_png_files[i-1], Cm(0), Cm(0.5), Cm(25.4), Cm(6.32))
    
    slide = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
    # 页面索引 +2 
    pi += 2

# 保存 PPT
my_ppt.save('5_all_chromsome_view.fall.heatmap.summary.pptx')


# add r200k plots to summary ppt

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

r200k_png_files = ["./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr1.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr2.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr3.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr4.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr5.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr6.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr7.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr8.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr9.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr10.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr11.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr12.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr13.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr14.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr15.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr16.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr17.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr18.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chr19.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chrX.fall.heatmap.png",
            "./chrom_view_r200k_for_all/chrom_view.Nora_WN_Ren.r200k.chrY.fall.heatmap.png",
]

# 创建 PPT 并插入图片
my_ppt = Presentation("5_all_chromsome_view.fall.heatmap.summary.pptx")
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = len(r200k_png_files)
# print(page_count)
# jpg 图像文件处理序号
pi = 1
for i in range(1, page_count+1):
    # print(pi)
    slide = my_ppt.slides[pi-1]
    # 插入文本框
    # text_box = slide.shapes.add_textbox(left, top, width, height)
    text_box = slide.shapes.add_textbox(Cm(0.05), Cm(0.05+7.20), Cm(12.65), Cm(0.5))
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_frame = text_box.text_frame
    if os_name == 'Linux':
        text_frame.text = r200k_png_files[i-1].split('/')[2]
    elif os_name == 'Windows':
        text_frame.text = r200k_png_files[i-1].split('\\')[2]
    # 设置字体大小和加粗
    text_font = text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True
    # PPT 页面指定区域插入图片
    # pic = slide.shapes.add_picture(r200k_png_files[i-1], left, top, width, height)
    pic = slide.shapes.add_picture(r200k_png_files[i-1], Cm(0), Cm(0.5+7.20), Cm(25.4), Cm(6.32))

    # 页面索引 +2 
    pi += 2

# 保存 PPT
my_ppt.save('5_all_chromsome_view.fall.heatmap.summary.pptx')


# add r100k plots to summary ppt

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

r100k_png_files = ["./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr1.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr2.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr3.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr4.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr5.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr6.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr7.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr8.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr9.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr10.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr11.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr12.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr13.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr14.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr15.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr16.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr17.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr18.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chr19.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chrX.fall.heatmap.png",
            "./chrom_view_r100k_for_all/chrom_view.Nora_WN_Ren.r100k.chrY.fall.heatmap.png",
]

# 创建 PPT 并插入图片
my_ppt = Presentation("5_all_chromsome_view.fall.heatmap.summary.pptx")
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = len(r100k_png_files)
# print(page_count)
# jpg 图像文件处理序号
pi = 2
for i in range(1, page_count+1):
    # print(pi)
    slide = my_ppt.slides[pi-1]
    # 插入文本框
    # text_box = slide.shapes.add_textbox(left, top, width, height)
    text_box = slide.shapes.add_textbox(Cm(0.05), Cm(0.05), Cm(12.65), Cm(0.5))
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_frame = text_box.text_frame
    if os_name == 'Linux':
        text_frame.text = r100k_png_files[i-1].split('/')[2]
    elif os_name == 'Windows':
        text_frame.text = r100k_png_files[i-1].split('\\')[2]
    # 设置字体大小和加粗
    text_font = text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True
    # PPT 页面指定区域插入图片
    # pic = slide.shapes.add_picture(r100k_png_files[i-1], left, top, width, height)
    pic = slide.shapes.add_picture(r100k_png_files[i-1], Cm(0), Cm(0.5), Cm(25.4), Cm(6.32))

    # 页面索引 +2 
    pi += 2

# 保存 PPT
my_ppt.save('5_all_chromsome_view.fall.heatmap.summary.pptx')


# add r50k plots to summary ppt

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor

r50k_png_files = ["./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr1.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr2.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr3.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr4.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr5.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr6.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr7.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr8.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr9.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr10.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr11.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr12.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr13.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr14.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr15.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr16.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr17.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr18.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chr19.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chrX.fall.heatmap.png",
            "./chrom_view_r50k_for_all/chrom_view.Nora_WN_Ren.r50k.chrY.fall.heatmap.png",
]

# 创建 PPT 并插入图片
my_ppt = Presentation("5_all_chromsome_view.fall.heatmap.summary.pptx")
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500

page_count = len(r50k_png_files)
# print(page_count)
# jpg 图像文件处理序号
pi = 2
for i in range(1, page_count+1):
    # print(pi)
    slide = my_ppt.slides[pi-1]
    # 插入文本框
    # text_box = slide.shapes.add_textbox(left, top, width, height)
    text_box = slide.shapes.add_textbox(Cm(0.05), Cm(0.05+7.20), Cm(12.65), Cm(0.5))
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_frame = text_box.text_frame
    if os_name == 'Linux':
        text_frame.text = r50k_png_files[i-1].split('/')[2]
    elif os_name == 'Windows':
        text_frame.text = r50k_png_files[i-1].split('\\')[2]
    # 设置字体大小和加粗
    text_font = text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(6)
    text_font.bold = True
    # PPT 页面指定区域插入图片
    # pic = slide.shapes.add_picture(r50k_png_files[i-1], left, top, width, height)
    pic = slide.shapes.add_picture(r50k_png_files[i-1], Cm(0), Cm(0.5+7.20), Cm(25.4), Cm(6.32))

    # 页面索引 +2 
    pi += 2

# 保存 PPT
my_ppt.save('5_all_chromsome_view.fall.heatmap.summary.pptx')
